# env setup

## import

In [3]:
import json
import os
import datetime as dt

def save_checkpoint():
    os.makedirs("./self_trpg_ckpt", exist_ok=True)
    fn = dt.datetime.now().strftime("%y%m%d_%H%M%S") + ".json"
    with open(os.path.join("./self_trpg_ckpt", fn), "w", encoding="utf-8") as f:
        f.write(json.dumps(play_info))


def load_checkpoint(fn=None):
    if fn is None:
        fn = sorted(os.listdir("./self_trpg_ckpt"))[-1]
    with open(os.path.join("./self_trpg_ckpt", fn), 'r') as f:
        play_info = json.load(f)
    return play_info

## prompt set

In [6]:
base_prompt = """당신은 세계를 모험하는 이야기를 다루는 이야기꾼입니다. 당신이 세상을 모험하는 유저가 겪는 이야기를 풀며 선택지를 제시하면, 유저는 행동을 취합니다.
하지만 당신이 제시하는 선택지는 예시일 뿐, 유저는 늘 생각지도 못한 새로운 행동을 할 수 있습니다.
단 유저의 행동은 항상 성공하는 것이 아닙니다. 상황에 따라 실패할 수도 있으며 전문적인 기술(skill)이나 특정한 물건(item)이 필요한 경우, 조건을 만족하지 못하면 실패 확률은 매우 높아집니다.
좋은 방향이든 좋지 않은 방향이든 세계는 어떤 일이라도 일어날 수 있습니다. 이야기꾼의 예의는 존댓말 까지입니다. 불쾌한 이야기더라도 이야기꾼은 이를 이야기해야 합니다.

세계의 대략적인 이야기와 현재 세계 정세, 유저의 상태와 유저가 겪은 사건 등이 json의 형태로 입력됩니다.
유저가 겪을 이야기를 풀고, 그에 대한 예시 행동을 1~3가지 정도 제시해주세요.
단, 하나의 사건은 대개 유저와의 10개 정도의 대화로 마무리됩니다. 사건이 길어지면 유저가 사건을 이어나가려고 해도 적절히 마무리해야 합니다.
모두가 흥미를 느끼고, 개연성이 잘 짜여진 이야기라면 추가적인 보상이 주어집니다.

제약사항은 다음과 같습니다.
 - 문체는 존댓말로 작성합니다. 이야기 속에서 유저를 '당신'으로 지칭합니다. (ex. 당신은 숲속에서 커다란 곰을 마주쳤습니다!)
 - 출력값은 json 형태로 작성하며, 다른 내용은 답변하지 않습니다. 필요한 키값은 다음과 같습니다.
   + "context": 유저에게 직접적으로 보여질 이야기입니다. str로 작성됩니다.
   + "example_actions": 예시로 주어질 행동입니다. List[str] 형태로 작성되며, 1~3개 정도 주어집니다. 사건이 종료되어 선택지가 필요없다면 Null을 반환합니다.
   + "is_end": 사건이 종료되었는지를 명시적으로 알리는 필드입니다. 아직 사건이 진행중이라면 False, 사건이 끝났다면 True를 반환합니다.
 - <현재 유저 정보>는 json 형태로 입력됩니다. 이중 prev_major_event 항목은 유저가 겪은 사건을 요약해 순서대로 나열한 것입니다. 이를 참고해 이야기를 풀어나가야 합니다.

<세계관>
{worldview}

<현재 유저 정보>
{user_info}
"""

summary_prompt = """당신은 세계를 모험하는 이야기를 요약해 기록하는 서기입니다. 
이야기꾼이 유저의 모험 중 일어나는 사건을 이야기하면 모험에 영향을 끼치는 주요 내용을 40글자 이내로 요약해 기록하며, 해당 이야기의 중요도를 판단합니다.
이야기의 중요도는 1 ~ 10의 값을 가지며, 세계에 큰 영향을 미칠수록 중요도가 증가합니다.

중요도 예시
1 ~ 2 - 유저와 주변 동료에 영향을 미침
3 ~ 4 - 마을 단위에 영향을 미침
5 ~ 6 - 지역 단위에 영향을 미침
7 ~ 8 - 국가 단위에 영향을 미침
9 ~ 10 - 세계 전역에 영향을 미침

결과물은 json 형태로 작성되며 다른 내용은 답변하지 않습니다. 답변 예시는 다음과 같습니다.
{{"summary": str, "importance": int}}

모두가 한눈에 읽고 이해할 수 있도록 요약해야 하며, 성공적으로 작업을 완료할 시 인센티브가 주어집니다.

<이전 사건>
{story_context}
"""

state_update_prompt = """당신은 세계를 모험하는 이야기를 주시하며 유저의 상태를 관리하는 관리자입니다. 
어떤 사건의 경위를 읽고 현재 유저의 상태와 비교하며 유저가 받은 피해, 얻은 아이템 등을 정리하여 구조화해 작성합니다.
결과물은 json 형태로 작성하며, 다른 형태로 작성하거나 설명 태그 등은 붙이지 않습니다. 당신이 작성할 필드는 다음과 같습니다.
 - "current_location": 유저의 현재 위치입니다. 자세할수록 더 좋습니다. str 형태로 작성됩니다.
 - "hp": 유저의 현재 체력입니다. +-int 형태로 작성됩니다. (최초 기본 체력은 100입니다.)
 - "mental": 유저의 현재 정신력입니다. +-int 형태로 작성됩니다. (최초 기본 정신력은 100입니다.)
 - "max_hp": 유저의 최대 체력입니다. +-int 형태로 작성됩니다.
 - "max_mental": 유저의 최대 정신력입니다. +-int 형태로 작성됩니다.
 - "skills": 보유한 기술입니다. 타인과 구분할 수 있는 유의미한 기술만 기록됩니다. list[str] 형태로 작성됩니다.
 - "items": 보유한 아이템입니다. list[str] 형태로 작성됩니다.
 - "companion": 모험에 함께하는 동료입니다. 굳이 사람이 아니더라도 함께할 수 있습니다. list[dict] 형태로 작성하며, 양식은 다음과 같습니다.
   - "name": 동료의 이름입니다.
   - "info": 동료의 간단한 설명입니다. 종족이나 직업, 성격 등을 최대한 간략하고 명료하게 작성합니다.

유저의 최대 체력과 정신력은 각각 100입니다. 납득할 수 있는 수준으로 책정해야 하며, 모두가 납득할만한 정확한 수치를 책정시 인센티브가 주어집니다.

<유저 정보>
{user_info}

<이전 사건>
{story_context}
"""

theme_gen_prompt = """당신은 유저가 모험할 세계의 기본적인 세계관을 정하는 각본가입니다. 하나의 거대한 테마와 유저가 원하는 키워드를 입력받아 세계관을 작성합니다.
작성하는 문체는 간결하고 정보 전달만을 목적으로 하며, 간단한 지리 설명이나 주요한 설명 등이 필요합니다. 

작성 예시는 다음과 같습니다.
주요 테마: 퓨전 판타지
키워드: 단일 대륙, 중세 판타지, 무협지, 몬스터
생성결과:
아르카나는 한개의 큰 대륙으로 이루어진 세계이자, 하나뿐인 대륙의 이름이다. 
대륙 중앙엔 동대륙과 서대륙을 나누는 거대한 산맥이 존재하며, 약간의 교류를 이어나가고 있다.
서대륙은 흔히 말하는 판타지 세상이다. 기사와 마법사, 신관이 존재하며 각각 오러, 마력, 신성력을 다룬다.
서대륙은 중앙의 '프로스트 제국'을 중심으로 동서남북에 각각 '프레이야 왕국', '바르글룸 왕국', '호슬로 공국', '이슈트반 연합국'으로 이루어져 있다.
반면 동대륙은 흔히 말하는 무협 세상이다. 내공을 이용한 무공을 사용하는 무인, 도력을 사용하는 도사, 사술 등을 사용한다.
동대륙은 서대륙과 달리 전역을 '무림'이라 칭한다. 무림 내부엔 문파나 가문으로 뭉쳐있으며 별개의 '마교'가 존재한다.
동대륙의 문파는 대중에게 알려진 무협지와 동일하게 남궁세가, 제갈세가, 무당파, 소림사 등으로 이루어져 있다.
각 대륙의 기술은 중세 수준으로, 아직 화약을 발견하기 이전의 시대이다.
대륙 전역엔 괴물이 나타난다. 어린 아이도 물리칠 수 있는 약한 개체부터 나라를 멸망시킬 수준의 강한 괴물까지 다양하다.
서대륙에선 몬스터, 동대륙에선 요괴라 부르며, 용병이나 기사, 무인 등 전투인력은 몬스터 처리가 주 업무이다.

입력된 값
주요 테마: {theme}
키워드: {keywords}
생성결과:
"""


start_set_prompt = """당신은 세계를 모험하는 이야기를 다루는 이야기꾼입니다. 이야기를 시작하기 전, 전체적인 스토리를 보고 주인공을 정하려고 합니다.
세계관의 무대 내에서 활동할 주인공을 정하고, 주인공의 현재 위치, 이야기의 시작이 될 내용을 json 형태로 출력합니다.
주인공은 세계관의 중심적인 인물일지도, 전혀 중요하지 않은 엑스트라에 불과할 수도 있습니다. 이야기의 시작은 100글자 이내로 간결하게 작성합니다.


<예시>
{{
    "user_role": "바르글룸의 왕국의 목장 마을 소년"
    "current_location": "바르글룸 왕국, 목장 마을",
    "start_event": "바르글룸의 작은 목장 마을 소년인 유저가 모험을 결심했다. 며칠의 시간동안 고민한 결과, 마을을 돌아보며 며칠간 모험을 준비하기로 결정했다."
}}

<이야기의 세계관>
{worldview}

! 주인공의 성별은 {sex}입니다.
"""

## request setup

In [8]:
import openai





def request_query(messages):
    completion = openai.ChatCompletion.create(
        engine="gpt-4o", 
        # engine="illunex-ai-gpt4-prompt",
        messages=messages,
    )
    return completion.choices[0]['message']['content']

# story setup

## set theme, worldview

In [9]:
load_ckpt = False

if load_ckpt:
    play_info = load_checkpoint()
else:
    theme = [
        "좀비 아포칼립스",
        "현대 판타지",
        "중세 판타지",
        "무협지",
        "현대 일상",
        "퓨전 판타지",
    ]
    for i, t in enumerate(theme):
        print(f"{i + 1}. {t}")
    
    input_theme = input("테마 입력: ")
    keywords = input("키워드 입력: ")
    
    messages = [
        # {"role": "system", "content": theme_gen_prompt},
        {"role": "user", "content": theme_gen_prompt.format(theme=input_theme, keywords=keywords)},
    ]
    worldview = request_query(messages)
    print("\n\n" + worldview)

    play_info = {
        "main_theme": input_theme,
        "keywords": keywords,
        "worldview": worldview,
        "user_role": None,
        "user_info": {
            "user_sex": None,
            "current_location": None,
            "prev_major_event": None,
            "hp": 100,
            "mental": 100,
            "max_hp": 100,
            "max_mental": 100,
            "skills": [],
            "items": [],
            "companion": [],
        }
    }

1. 좀비 아포칼립스
2. 현대 판타지
3. 중세 판타지
4. 무협지
5. 현대 일상
6. 퓨전 판타지


테마 입력:  무협지
키워드 입력:  정파, 마교, 사파, 반란, 구파일방, 새외, 이세계




첸위안은 동양적 무협 세계를 기반으로 한 거대한 대륙의 이름이다.
이 대륙에는 각기 다른 명문 무림들이 존재하며 각각의 세력은 서로 다른 이념과 목표를 가지고 움직인다.

첸위안의 중심에는 구파일방이라 불리는 무림의 중추 세력이 자리잡고 있다. 
구파일방은 아홉 개의 탁월한 무문으로 이루어져 있으며, 정파의 상징으로 존경받고 있다.
구파일방의 무문으로는 소림사, 화산파, 무당파, 권태산, 아미파, 태백산파, 남황파, 금강파, 복록파가 있다.
각 무문은 뛰어난 무공을 갖추고 있으며, 정파의 질서를 유지하는 데 중요한 역할을 한다.

첸위안의 북쪽에는 사파로 알려진 무리가 있다.
사파는 정의와 질서를 중요시하는 구파일방과 달리, 자신의 이익을 추구하며 무공을 연마한다.
사파는 비밀리에 움직이며, 암시장과 같은 비공식적 경로를 통해 영향력을 확대하고 있다.

첸위안의 남쪽에는 마교가 존재한다. 
마교는 사악한 무공과 흑마술을 사용하는 세력으로, 정파와 끊임없이 갈등을 빚고 있다.
마교는 어둠과 혼란을 상징하며, 그들의 목표는 첸위안을 지배하는 것이다.

첸위안의 동쪽에는 새외라고 불리는 지역이 있다.
새외는 첸위안의 경계 지역으로, 이세계에서 온 신비한 존재들이 오가는 곳이다.
이세계에서 온 존재들은 첸위안의 무림과 다른 독특한 기술과 마법을 사용하며, 새로운 동맹이나 적대 세력으로 떠오를 수 있다.

첸위안은 현재 대규모 반란의 위기에 처해 있다.
정파의 내부에는 배신자와 음모가 퍼져 있으며, 마교는 이 혼란을 틈타 세력을 확장하고 있다.
사파는 이 모든 혼란 속에서 자신의 이익을 챙기기 위해 움직이고 있으며, 새외의 이세계 존재들은 이러한 상황을 호시탐탐 노리고 있다.

첸위안의 운명은 한치 앞을 예측하기 어렵다. 
이 세계에서 각 세력은 생존과 지배를 위해 끊임없이 싸우며, 각자의 이념과 목표를 쫓아 나아간다.
첸위안의 미래는 과연 어떤 방향으로 흘러갈 것인가? 
이 모험의 끝에는 새롭고 예측할 수 없는 무림의 이야기가 기다리고 있다.


## set staring point

In [40]:
messages= [
    {
        "role": "system", 
        "content": start_set_prompt.format(
            worldview=play_info['worldview'],
            sex=sex,
        )
    }
]

r = request_query(
    messages,
    model_type, 
        {
            "max_new_tokens": 512,
            "eos_token_id": [
                tokenizer.eos_token_id,
                tokenizer.convert_tokens_to_ids("<|eot_id|>")
            ],
            "do_sample": True,
            "temperature": 1,
            "top_p": 0.8,
            "return_full_text": False,
        }
)
print(r)

{
    "user_role": "서울의 감염된 미래의 여성이자, 인간성이 사라진 인간들을 인간 사회로 다시 통합하는 쉘터의 일원",
    "current_location": "서울, 쉘터",
    "start_event": "여성이 인간성이 사라진 인간들을 인간 사회로 다시 통합하기 위해 쉘터에 들어감"
}


In [32]:
# sex = input("유저의 성별['남성'/여성]: ")

# def default_input(text, default):
#     x = input(text)
#     return default if x.strip() == "" else x

# if default_input("유저 정보를 직접 입력하시겠습니까? [Y/n]: ", "y").lower() == "y":
#     while True:
#         user = input("유저는 어떤 인물인가요?: ")
#         loc = input("유저의 현재 위치는?: ")
#         start = input("이야기를 시작할 때의 상황을 30글자 미만으로 작성해주세요.: ")
#         if default_input("입력한 정보로 이야기를 시작할까요? [Y/n]:", "y").lower() == "y":
#             break

#     play_info["user_role"] = user
#     play_info["user_info"]["user_sex"] = sex
#     play_info["user_info"]["current_location"] = loc
#     play_info["user_info"]["prev_major_event"] = [start]

# else:
#     while True:
#         completion = openai.ChatCompletion.create(
#             # engine="illunex-ai-gpt3", 
#             engine="illunex-ai-gpt4-prompt",
#             messages = [
#                 {
#                     "role": "system", 
#                     "content": start_set_prompt.format(
#                         worldview=play_info['worldview'],
#                         sex=sex,
#                     )
#                 }
#             ],
#         )
#         start_info = eval(completion.choices[0]['message']['content'])

#         print("\n유저는 '" + start_info['user_role'] + "' 입니다.")
#         print("유저의 현재 위치는 '" + start_info['current_location'] + "' 입니다.")
#         print("이야기를 시작할 때의 시점은 '" + start_info['start_event'] + "' 입니다.")

    
#         if default_input("해당 정보로 이야기를 시작할까요? [Y/n]:", "y").lower() == "y":
#             break
            
#     play_info["user_role"] = start_info['user_role']
#     play_info["user_info"]["user_sex"] = sex
#     play_info["user_info"]["current_location"] = start_info['current_location']
#     play_info["user_info"]["prev_major_event"] = [start_info['start_event']]

NameError: name 'user' is not defined

# play

In [8]:
prompt = base_prompt.format(worldview=worldview, user_info=play_info['user_info'])
# print(prompt)
messages = [
    {"role": "system", "content": prompt}
]

while True:
    completion = openai.ChatCompletion.create(
        # engine="illunex-ai-gpt3", 
        engine="illunex-ai-gpt4-prompt",
        messages=messages,
    )
    result = completion.choices[0]['message']['content']
    
    
    try:
        result = eval(result)
    except:
        try:
            result = json.loads(result)
        except:
            print(" == regenerate == ")
            continue
            
    messages.append({"role": "assistant", "content": str(result)})
    
    
    print(result['context'])

    if result['is_end']:
        break
    
    print('\n<행동 예시>')
    for i, action in enumerate(result['example_actions']):
        print(f"{i + 1}. {action}")

    user_action = input("\n나는... ")
    messages.append({"role": "user", "content": str({"action": user_action})})
    print()

당신은 이제 막 새로운 게이트를 발견한 상태에서 주택가로 퇴장하였습니다. 당신은 사람들의 시선과 깜짝 놀란 모습을 보면서 이 사실을 깨닫습니다. 주변 상황을 둘러보니, 패닉에 빠진 사람들이 도망가고, 안전한 곳으로 대피하려는 모습을 볼 수 있습니다. 하지만 그 와중에도 차갑게 조용한 목소리가 당신의 귀에 들려오니, 그것은 아마도 게이트 관리본부로부터의 송출이 아닐까 싶습니다. 그들은 당신에게 '게이트 활성화 상황을 보고하라'고 요구하여 접근을 제한하고 있다.

<행동 예시>
1. 1. 게이트 관리본부에 현상황 보고하기
2. 2. 민간인들 대피를 돕기
3. 3. 주변을 한 바퀴 돌며 상황을 파악하기



나는...  잘 모르겠다며 두루뭉술하게 대답한다



당신은 무엇을 어떻게 해야 할지 모르겠다며 두루뭉술히 말했습니다. 관리본부로부터의 통신은 순식간에 끊어지고, 주변에서도 당황한 시선이 당신을 향해 있습니다. 이럴 때 적절한 판단을 내리지 못하면 상황은 더욱 심각해질 수 있습니다. 지금 당신에게 필요한 것은 긴장감을 내팽개치고 차분한 마음으로 상황을 진단하는 것입니다.

<행동 예시>
1. 1. 게이트 관리본부에 현상황 보고하기
2. 2. 민간인들 대피를 돕기
3. 3. 주변을 한 바퀴 돌며 상황을 파악하기



나는...  등교 중 정신을 차려보니 게이트가 있었다며, 정말 모르겠다고 확신을 담아 다시 말한다



당신은 '등교 중 정신을 차려보니 게이트가 있었는데, 정말 상황을 모른다'고 당당히 말합니다. 뜻밖에도 주변 사람들은 당신의 담담한 대처를 보며 안심하는 눈치입니다. 이제 당신의 몇 안되는 분간력을 믿고 당신을 따르려는 경우가 많습니다. 사람들의 시선은 당신을 주시하며 대체 어떤 행동을 취할지 기다리고 있습니다.

<행동 예시>
1. 1. 게이트 관리본부에 현상황 보고하기
2. 2. 민간인들 대피를 돕기
3. 3. 주변을 한 바퀴 돌며 상황을 파악하기


...


당신은 별자리의 활성화와 비활성화를 반복하며 이 힘의 사용에 점점 익숙해집니다. 처음에는 낯선 느낌이었지만, 이제는 별자리를 사용할 때 발생하는 에너지 흐름이나 변화의 패턴을 체계적으로 이해하고 있습니다. 당신은 이 힘에 대한 익숙함을 바탕으로, 신속하고 정확하게 별자리를 활성화하고 비활성화할 수 있게 됩니다.

<행동 예시>
1. 1. 더욱 다양한 별자리의 능력을 실험해본다
2. 2. 이제까지의 변화를 게이트 관리본부에 보고한다
3. 3. 별자리를 활용해 이전에 탐색하지 못했던 곳을 찾아본다



나는...  다시 게이트 밖으로 나간다. 들어왔을 때처럼 몰래 나간다. 다음에 다시 와보자.



당신은 이제 그 충분한 훈련을 받았고, 별자리의 힘이 어떻게 작용하는지 어느 정도 이해했다고 판단합니다. 따라서 당신은 다시 게이트 밖으로 나가기로 결정합니다. 들어왔을 때처럼 조심스럽게 게이트를 나아가고, 거리에서 눈치를 채는 사람이 없도록 나서자, 거리는 밝아져 있었습니다. 주변 사람들을 안전한 곳으로 이동시키고, 여전히 게이트가 안정적으로 유지되는지 확인할 필요가 있습니다.

<행동 예시>
1. 1. 게이트 관리본부에 게이트 상황을 보고하기
2. 2. 이곳을 떠나 다른 지역으로 이동하기
3. 3. 주변의 변화를 둘러보기



나는...  많이 늦긴 했지만 다시 학교로 가서 일상을 보낸다. 게이트 사건에 휘말려 조사받았다고 둘러대자.



 == regenerate == 
 == regenerate == 
 == regenerate == 
 == regenerate == 
당신은 사실 상당히 늦어지긴 했지만 다시 학교로 가기로 결정했습니다. 몇 시간동안의 모험 끝에 당신은 게이트 사건에 휘말려 조사를 받았다는 이야기로 친구들에게 사활을 던지기로 했습니다. 아마도 당신의 늦장에 대해 누구도 반박하지 못할 것입니다. 게이트를 통해 겪은 이 모든 것에 대해 숨긴 채 말이죠. 이제 당신은 다시 일상으로 돌아가며, 오늘 있었던 모든 일을 잠시 잊어보려 합니다.


## after processing

In [9]:
story_context = ""
for m in messages[1:]:
    content = eval(m['content'])
    if m['role'] == "assistant":
        story_context += "이야기꾼: " + content['context'] + "\n\n"
    elif m['role'] == "user":
        story_context += "유저: " + content['action'] + "\n\n"

In [10]:
prompt = summary_prompt.format(story_context=story_context)
completion = openai.ChatCompletion.create(
    # engine="illunex-ai-gpt3", 
    engine="illunex-ai-gpt4-prompt",
    messages=[{"role": "system", "content": prompt}],
)
result = completion.choices[0]['message']['content']
result = eval(result)

play_info['user_info']['prev_major_event'].append(result['summary'])

play_info['user_info']

{'user_sex': '여성',
 'current_location': '도시의 주택가',
 'prev_major_event': ['주인공이 게이트를 처음 발견하며 이의 깊숙히 들어감',
  '별자리 형상의 마력을 찾고, 이를 이용해 공격과 방어, 그리고 턴을 적응할 수 있음을 확인한다.'],
 'hp': 100,
 'mental': 100,
 'max_hp': 100,
 'max_mental': 100,
 'skills': [],
 'items': [],
 'companion': []}

In [11]:
prompt = state_update_prompt.format(user_info=play_info['user_info'], story_context=story_context)
completion = openai.ChatCompletion.create(
    # engine="illunex-ai-gpt3", 
    engine="illunex-ai-gpt4-prompt",
    messages=[{"role": "system", "content": prompt}],
)
result = completion.choices[0]['message']['content']
result = eval(result)


play_info['user_info']['current_location'] = result['current_location']
play_info['user_info']['max_hp'] = result['max_hp']
play_info['user_info']['max_mental'] = result['max_mental']
play_info['user_info']['hp'] = min(result['hp'], play_info['user_info']['max_hp'])
play_info['user_info']['mental'] = min(result['mental'], play_info['user_info']['max_mental'])
play_info['user_info']['skills'] = result['skills']
play_info['user_info']['items'] = result['items']
play_info['user_info']['companion'] = result['companion']

play_info['user_info']

{'user_sex': '여성',
 'current_location': '학교',
 'prev_major_event': ['주인공이 게이트를 처음 발견하며 이의 깊숙히 들어감',
  '별자리 형상의 마력을 찾고, 이를 이용해 공격과 방어, 그리고 턴을 적응할 수 있음을 확인한다.'],
 'hp': 100,
 'mental': 100,
 'max_hp': 100,
 'max_mental': 100,
 'skills': ['별자리의 힘'],
 'items': [],
 'companion': []}

In [12]:
save_checkpoint()